In [1]:
! pip install transformers==4.25.1 fugashi ipadic mecab-python3 unidic-lite

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 33.8 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.3/613.3 kB 17.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 51.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.5/581.5 kB 10.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 16.5 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 121.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 kB 43.2 MB/s eta 0:00:00
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556704 sha256=3aa128d67490708fbc320698dd4b4f8365de9a335e9811ce0dc6bf33c6c63f70
  Stored in directory: /home/ec2-user/.cache/pip/wheels/4a/fb/28/21aa4bc5d87

# Token classification taskためのlabel作成

In [2]:
import torch
from transformers import BertJapaneseTokenizer
import pickle
import pandas as pd
import unicodedata

**tohoku tokenizer**

In [3]:
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-large-japanese')#cl-tohoku/bert-base-japanese-whole-word-masking

In [4]:
text1='ＧＲＡＤＥ２'
print(tokenizer.tokenize(text1))
text2='GRADE2'
print(tokenizer.tokenize(text2))

['GR', '##ADE', '2']
['GR', '##ADE', '2']


**load data**

In [5]:
with open('input_data/df_row_new_2.pkl', 'rb') as file:
    df_row=pickle.load(file)

**get tokens**

In [6]:
df_row['tokens'] = df_row['row'].apply(lambda x: tokenizer.tokenize(x))
df_row['pick_word_tokens'] = df_row['pick_words_new'].apply(lambda x: [tokenizer.tokenize(p) for p in x])
df_row[['row','tokens','pick_words_new','pick_word_tokens']]

,row,tokens,pick_words_new,pick_word_tokens
0,10回目、DAY245、爪囲炎改善したが再増悪。,"[10, 回, 目, 、, DAY, 24, ##5, 、, 爪, ##囲, 炎, 改善, ...",[爪囲炎],"[[爪, ##囲, 炎]]"
1,18時頃突然の前胸部痛と冷汗あり。18:15頃に意識消失、CPR(心肺蘇生法)開始。酸素投与...,"[18, 時, 頃, 突然, の, 前, 胸部, 痛, と, 冷, ##汗, あり, 。, ...",[心室細動],"[[心, ##室, 細, ##動]]"
2,1コース目D21、GRADE2好中球減少発現。,"[1, コース, 目, D, 21, 、, GR, ##ADE, 2, 好, ##中, 球,...",[GRADE2好中球減少],"[[GR, ##ADE, 2, 好, ##中, 球, 減少]]"
3,1コース目はシスプラチン投与の際のハイドレーションに伴い、一時的に体重増加(+3~4KG)、...,"[1, コース, 目, は, シス, ##プラ, ##チン, 投与, の, 際, の, ハイ...","[体重増加, 下肢浮腫]","[[体重, 増加], [下, ##肢, 浮, ##腫]]"
4,1コース目カルボプラチン(AUC5)、エトポシド(100MG/M2)にてG4の好中球減少、F...,"[1, コース, 目, カルボ, ##プラ, ##チン, (, A, ##UC, 5, ),...","[G4の好中球減少, 発熱性好中球減少症]","[[G, 4, の, 好, ##中, 球, 減少], [発熱, 性, 好, ##中, 球, ..."
...,...,...,...,...
7538,転倒、頭部を打撲、右眼窩外側壁骨折、顔面挫創、糖尿、アラニンアミノトランスフェラーゼ増加、高...,"[転倒, 、, 頭部, を, 打, ##撲, 、, 右, 眼, ##窩, 外, 側, ##壁...","[転倒, 頭部を打撲, 右眼窩外側壁骨折, 挫創, 糖尿, アラニンアミノトランスフェラーゼ...","[[転倒], [頭部, を, 打, ##撲], [右, 眼, ##窩, 外, 側, ##壁,..."
7539,右側腹部痛と下腹部痛みあり。既往歴に憩室炎があるが、今は問題ない。,"[右側, 腹部, 痛, と, 下, ##腹, 部, 痛み, あり, 。, 既, ##往, 歴...","[側腹部痛, 下腹部痛]","[[側, 腹部, 痛], [下, ##腹, 部, 痛]]"
7540,貧血進行なし。リムパーザ錠休薬期間もあったが腫瘍マーカーは下降続けた。,"[貧, ##血, 進行, なし, 。, リム, ##パー, ##ザ, 錠, 休, ##薬, ...",[腫瘍マーカー],"[[腫瘍, マー, ##カー]]"
7541,鑑別診断:潰瘍性大腸炎。,"[鑑, ##別, 診断, :, 潰, ##瘍, 性, 大腸, 炎, 。]",[大腸炎],"[[大, 腸, ##炎]]"


**create labels**

label_list=[0,1,2]
<br>
    0==>  "0" indicates the token doesn’t correspond to any entity.
<br>
    1==>  "B" indicates the beginning of an entity.
<br>
    2==>  "I" indicates a token is contained inside the same entity

In [7]:
def index_func(token,pick_word_tokens):
    flatten = lambda x: [y for l in x for y in flatten(l)] if type(x) is list else [x]
    pick_word_tokens_flatten=flatten(pick_word_tokens)    

    label=[0 for i in range(len(token))]
    label_index=[i for i in range(len(token)) if token[i] in pick_word_tokens_flatten]
    label=[1 if i in label_index  else 0 for i in range(len(label))]
    index=[]
    subindex=[]
    for i in range(len(label)):
        if label[i]==1:
            subindex.append(i)
        if label[i]==0:
            index.append(subindex)
            subindex=[]
    index= list ( filter ( None , index))    

    # remove label_index中の'が' 'の' などのsingle index sublist   e.g.[['筋炎','の','可能性'].['の']]
    remove=['が','の'] 
    index_filter=[]
    for subindex in index:
        if len(subindex)==1 and token[subindex[0]] in remove:
            continue
        else:
            index_filter.append(subindex)
    
    return index_filter

def label_func(index_list, tokens, pick_word):
    label=[0 for i in range(len(tokens))]
    for index in index_list:
            for i, idx in enumerate(index):
                if i == 0:
                    label[idx] = 1
                else:
                    label[idx] = 2      
    return label

In [8]:
df=df_row.copy()

In [9]:
df['index'] = df.apply(lambda x:  index_func(x['tokens'],x['pick_word_tokens']) if x['pick_word_tokens']!=['記載なし'] else 0 , axis=1)
df['labels'] = df.apply(lambda x: label_func(x['index'],x['tokens'],x['pick_words_new']), axis=1)

**問題点**

row token != pick_work_token
<br>
G3薬剤性大腸炎発現==>['G', '3', '薬剤', '性', '大腸', '炎', '発現', '。']
<br>
大腸炎==>['大', '腸', '##炎']
<br>
'大腸', '炎' != '大', '腸', '##炎'

In [9]:
df[df['row']=='G3薬剤性大腸炎発現。']

,level_0,file_name,management_number,row,AE,LLT,PT,pick_words_new,tokens,pick_word_tokens,index,labels
161,874,./input\イミフィンジ_20220101-0430.xlsx,202032679,G3薬剤性大腸炎発現。,[薬剤性大腸炎],['大腸炎'],['大腸炎'],[大腸炎],"[G, 3, 薬剤, 性, 大腸, 炎, 発現, 。]","[[大, 腸, ##炎]]",[],"[0, 0, 0, 0, 0, 0, 0, 0]"


In [10]:
print('pick_word',df[df['row']=='G3薬剤性大腸炎発現。']['pick_words_new'].item())
print('pick_word_tokens',df[df['row']=='G3薬剤性大腸炎発現。']['pick_word_tokens'].item())
print('row',df[df['row']=='G3薬剤性大腸炎発現。']['row'].item())
print('tokens',df[df['row']=='G3薬剤性大腸炎発現。']['tokens'].item())
print('labels',df[df['row']=='G3薬剤性大腸炎発現。']['labels'].item())
print('index',df[df['row']=='G3薬剤性大腸炎発現。']['index'].item())

pick_word ['大腸炎']
pick_word_tokens [['大', '腸', '##炎']]
row G3薬剤性大腸炎発現。
tokens ['G', '3', '薬剤', '性', '大腸', '炎', '発現', '。']
labels [0, 0, 0, 0, 0, 0, 0, 0]
index []


In [10]:
"""
この問題がある行を抽出
"""
flatten = lambda x: [y for l in x for y in flatten(l)] if type(x) is list else [x]
index_list=[]
for index,row in df.iterrows():
    #print(row['row'])
    #if row['pick_word']!= ['記載なし']:
        pick_word_token=row['pick_word_tokens']
        token=row['tokens']
        pick_word_token_flatten=flatten(pick_word_token) 
        for i in pick_word_token_flatten:
            u=set([i]) & set(token)
            if len(u) == 0:
                #print(i)
                #print(index)
                index_list.append(row['level_0'])
index_list=list(set(index_list))

In [11]:
df_ = df[df['level_0'].isin(index_list)]
df= df[~df['level_0'].isin(index_list)]
df_[['tokens','pick_words_new','pick_word_tokens','labels']].head(5)

,tokens,pick_words_new,pick_word_tokens,labels
128,"[G, 1, 低, アル, ##ブ, ##ミン, 血, 症, (, 重, ##篤, で, な...","[低アルブミン血症, クレアチニン増加, 低ナトリウム血症, 高カリウム血症, 高マグネシウ...","[[低, アル, ##ブ, ##ミン, 血, 症], [クレア, ##チ, ##ニン, 増加...","[0, 0, 1, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, ..."
161,"[G, 3, 薬剤, 性, 大腸, 炎, 発現, 。]",[大腸炎],"[[大, 腸, ##炎]]","[0, 0, 0, 0, 0, 0, 0, 0]"
262,"[GR, ##ADE, 3, 白, 血, ##球, 減少, 、, GR, ##ADE, 2,...","[白血球減少, 好中球数減少症, 貧血, 血小板数減少, 低アルブミン血症, リンパ球数減少...","[[白, 血, ##球, 減少], [好, ##中, 球, ##数, 減少, 症], [貧,...","[0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 0, 1, 2, 2, 2, ..."
540,"[下, ##痢, 回数, は, 改善, 傾向, で, あり, 食事, 再開, 、, 大腸, ...",[大腸炎],"[[大, 腸, ##炎]]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
708,"[大腸, 内, ##視, 鏡, 検査, 所, ##見, :, 大腸, 全体, 、, 特に, ...",[大腸炎],"[[大, 腸, ##炎]]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


**この問題を解決**

In [12]:
def label_func_2(row, tokens, pick_word_index_list):

    df = pd.DataFrame({'tokens':tokens})
    df.tokens = df.tokens.apply(lambda x: unicodedata.normalize("NFKC", x.replace('##', '')))
    tokens_index = []
    for t in df.tokens:
        tokens_index.append((row.find(t), row.find(t)+len(t)))
        row = row.replace(t, ''.join(['@' for i in range(len(t))]), 1)

    df['index'] = tokens_index
    for word_idx in pick_word_index_list:
        df['label'] = df.apply(lambda x: 2 if (x['index'][1]>word_idx[0]) and (x['index'][0]<word_idx[1]) else 0, axis=1)
        df['label'] = df.apply(lambda x: 1 if (x['label']==2) and (x['index'][0]<=word_idx[0]) and (word_idx[0]<x['index'][1]) else x['label'], axis=1)
    return df


df_t = df_
#df_t=df_.copy()
df_t['pick_word_index_list'] = df_.apply(lambda x: [(x['row'].find(w), x['row'].find(w)+len(w)) for w in x['pick_words_new']], axis=1)
df_t['df_label'] = df_t.apply(lambda x: label_func_2(x['row'], x['tokens'], x['pick_word_index_list']), axis=1)
df_t['labels'] = df_t['df_label'].apply(lambda x: x['label'].to_list())

In [13]:
display(df_t[['row','tokens','pick_words_new','labels']].head(3))

,row,tokens,pick_words_new,labels
128,G1低アルブミン血症(重篤でない)、G2クレアチニン増加(重篤でない)、G1低ナトリウム血症...,"[G, 1, 低, アル, ##ブ, ##ミン, 血, 症, (, 重, ##篤, で, な...","[低アルブミン血症, クレアチニン増加, 低ナトリウム血症, 高カリウム血症, 高マグネシウ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
161,G3薬剤性大腸炎発現。,"[G, 3, 薬剤, 性, 大腸, 炎, 発現, 。]",[大腸炎],"[0, 0, 0, 0, 1, 2, 0, 0]"
262,GRADE3白血球減少、GRADE2好中球数減少症、GRADE1貧血、GRADE1血小板数減...,"[GR, ##ADE, 3, 白, 血, ##球, 減少, 、, GR, ##ADE, 2,...","[白血球減少, 好中球数減少症, 貧血, 血小板数減少, 低アルブミン血症, リンパ球数減少...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [14]:
#元のデータにマージする
df=pd.concat([df, df_t],axis=0)

**bert input**

In [15]:
MAX_LENGTH =128  #512

# remove length>MAX_LENGTH
df['input_lengths']=df['tokens'].apply(lambda x:len(x))
df=df[df['input_lengths']<MAX_LENGTH-1]

# input token ids
## tokens add '[CLS]' [SEP]' 
df['tokens']=df['tokens'].apply(lambda x: ['[CLS]']+x+['[SEP]'])
df['input_ids']=df['tokens'].apply(lambda x:tokenizer.convert_tokens_to_ids(x))

# labels
# [CLS],[SEP] ==> -100 
df['labels']=df['labels'].apply(lambda x: [-100]+x+[-100])

# attention_mask
df['attention_mask']=df['tokens'].apply(lambda x:[1 for i in range(len(x))])

In [16]:
df[['row','tokens','pick_words_new','pick_word_tokens','labels']]

,row,tokens,pick_words_new,pick_word_tokens,labels
0,10回目、DAY245、爪囲炎改善したが再増悪。,"[[CLS], 10, 回, 目, 、, DAY, 24, ##5, 、, 爪, ##囲, ...",[爪囲炎],"[[爪, ##囲, 炎]]","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, ..."
1,18時頃突然の前胸部痛と冷汗あり。18:15頃に意識消失、CPR(心肺蘇生法)開始。酸素投与...,"[[CLS], 18, 時, 頃, 突然, の, 前, 胸部, 痛, と, 冷, ##汗, ...",[心室細動],"[[心, ##室, 細, ##動]]","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,1コース目D21、GRADE2好中球減少発現。,"[[CLS], 1, コース, 目, D, 21, 、, GR, ##ADE, 2, 好, ...",[GRADE2好中球減少],"[[GR, ##ADE, 2, 好, ##中, 球, 減少]]","[-100, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, ..."
3,1コース目はシスプラチン投与の際のハイドレーションに伴い、一時的に体重増加(+3~4KG)、...,"[[CLS], 1, コース, 目, は, シス, ##プラ, ##チン, 投与, の, 際...","[体重増加, 下肢浮腫]","[[体重, 増加], [下, ##肢, 浮, ##腫]]","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,1コース目カルボプラチン(AUC5)、エトポシド(100MG/M2)にてG4の好中球減少、F...,"[[CLS], 1, コース, 目, カルボ, ##プラ, ##チン, (, A, ##UC...","[G4の好中球減少, 発熱性好中球減少症]","[[G, 4, の, 好, ##中, 球, 減少], [発熱, 性, 好, ##中, 球, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...
7446,40代女性。既往歴にくも膜下出血があり、下肢に神経障害性疼痛あり。初回BRCA陽性にてリムパ...,"[[CLS], 40, 代, 女性, 。, 既, ##往, 歴, に, く, ##も, ##...","[膜下出血, 神経障害性疼痛, 慢性痛]","[[膜, 下, 出血], [神経, 障害, 性, 疼, ##痛], [慢性, 痛]]","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7461,下顎骨転移発現。,"[[CLS], 下, 顎, ##骨, 転移, 発現, 。, [SEP]]",[骨転移],"[[骨, 転移]]","[-100, 0, 0, 1, 2, 0, 0, -100]"
7532,補足:AV55CMからAV40CM、AV25CMから30CM、AV20CMからAV8CMに全...,"[[CLS], 補足, :, AV, 55, CM, から, AV, 40, CM, 、, ...",[大腸炎],"[[大, 腸, ##炎]]","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7539,右側腹部痛と下腹部痛みあり。既往歴に憩室炎があるが、今は問題ない。,"[[CLS], 右側, 腹部, 痛, と, 下, ##腹, 部, 痛み, あり, 。, 既,...","[側腹部痛, 下腹部痛]","[[側, 腹部, 痛], [下, ##腹, 部, 痛]]","[-100, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 0, 0, ..."


**save**

In [17]:
with open('input_data/df_row_token_classification_dataset_2.pkl', 'wb') as file:
    pickle.dump(df, file)